<a href="https://www.kaggle.com/code/shedai/lmm-reasoning-augmentation-2-tree-of-thought-tot?scriptVersionId=250143010" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 2. Tree of Thoughts (TOT) Kavramı ve Karşılaştırma 

Standart (direct): Sadece “Solution:” prompt’u ile doğrudan birkaç cevap alıyoruz.

Chain-of-Thought (CoT): Prompt’a “Let’s think step by step…” ekleyerek modelin ara düşünce adımlarını vermesini sağlıyoruz.

Tree-of-Thoughts (ToT): Önce kısa “düşünce önerileri” (branches) üretiyor, sonra her kol için son cevabı alıyoruz.

In [2]:
!pip install transformers sentencepiece --quiet

from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

device = "cpu"  # GPU yerine CPU kullanarak bellek tüketimini düşürüyoruz


2025-07-12 16:05:25.671875: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752336325.911901      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752336325.979435      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:

model_name = "google/flan-t5-large"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name).to(device)

# Problem tanımı
problem = (
    "You have 3 cups, each with 4 stones. "
    "Goal: One cup has 6 stones, others have 3 each. "
    "You can move one stone at a time between any two cups. "
    "Plan the sequence of moves."
)

# Standart yaklaşım
prompt_standard = f"Problem: {problem}\nSolution:"
inputs_std = tokenizer.encode(prompt_standard, return_tensors="pt")
# DÜZELTME: max_length yerine max_new_tokens kullanıldı.
outs_std = model.generate(inputs_std, max_new_tokens=150, num_return_sequences=4, do_sample=True, top_k=50)
solutions_std = [tokenizer.decode(o, skip_special_tokens=True).replace(prompt_standard, "").strip() for o in outs_std]

# Tree of Thoughts yaklaşımı
def expand(prefix):
    prompt = f"Problem: {problem}\nThoughts so far: {prefix}\nNext thought:"
    inputs = tokenizer.encode(prompt, return_tensors="pt")
    # DÜZELTME: max_length yerine max_new_tokens kullanıldı.
    outs = model.generate(inputs, max_new_tokens=100, num_return_sequences=2, do_sample=True, top_k=50)
    return [tokenizer.decode(o, skip_special_tokens=True).split("Next thought:")[-1].strip() for o in outs]

paths = []
for t1 in expand(""):
    for t2 in expand(t1):
        paths.append([t1, t2])

solutions_tot = []
for path in paths:
    prefix = " | ".join(path)
    prompt = f"Problem: {problem}\nThought path: {prefix}\nFinal solution:"
    inputs = tokenizer.encode(prompt, return_tensors="pt")
    # DÜZELTME: max_length yerine max_new_tokens kullanıldı.
    out = model.generate(inputs, max_new_tokens=150, do_sample=True, top_k=50)
    sol = tokenizer.decode(out[0], skip_special_tokens=True).split("Final solution:")[-1].strip()
    solutions_tot.append(sol)

# Doğruluk kontrolü
def valid(sol):
    return ("6 stones" in sol) and ("3 stones" in sol)

# Sonuçların yazdırılması
print("=== Standart Yaklaşım Çözümleri ===")
for i, s in enumerate(solutions_std, 1):
    print(f"{i}. {s}  -> Geçerli: {valid(s)}")

print("\n=== Tree of Thoughts Çözümleri ===")
for i, s in enumerate(solutions_tot, 1):
    print(f"{i}. {s}  -> Geçerli: {valid(s)}")

# Karşılaştırma özeti
print(f"\nStandart geçerli çözümler: {sum(valid(s) for s in solutions_std)}/{len(solutions_std)}")
print(f"Tree of Thoughts geçerli çözümler: {sum(valid(s) for s in solutions_tot)}/{len(solutions_tot)}")

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

=== Standart Yaklaşım Çözümleri ===
1. One cup has 12 stones, so move the stone from that cup to the other cup, which has 6 stones and then 3 more. Make sure two cups have 6 stones. Move stones one by one between 1st and 3rd cups. Make sure the cups have 6 stones and 6 more stones. Move stones one by one between 2nd and 3rd cups. Make sure two cups have 6 stones and 6 more stones. Move stones one by one between 2nd and 2nd cups. Make sure that two cups have 6 stones and 6 more stones. Move stones one by one between 2nd and 2nd cups. Make sure the cups have 6 stones and 6 more stones. Move stones one by one between 1st and 2  -> Geçerli: False
2. Every cup has 4 stones. You can move one between any of the 3 cups. Since you start with 1 stone, repeat this process as you go.  -> Geçerli: False
3. If you have a quarter of a cup, then move four stones to the second cup. Take the diamond, move it between the four cups in order to get another quarter. Take the 3rd stone, move it between the t

In [4]:

problem = (
    "You have 3 cups, each with 4 stones. "
    "Goal: One cup has 6 stones, others have 3 each. "
    "You can move one stone at a time between any two cups. "
    "Plan the sequence of moves."
)

# Ayarlar
# DÜZELTME: Tüm 'max_length' anahtarları 'max_new_tokens' olarak değiştirildi.
# Değerler, sadece üretilecek metnin uzunluğunu yansıtacak şekilde ayarlandı.
standard_kwargs = {"max_new_tokens": 50, "num_return_sequences": 4, "do_sample": True, "top_k": 50}
cot_kwargs      = {"max_new_tokens": 80, "num_return_sequences": 4, "do_sample": True, "top_k": 50}
tot_expand_kwargs = {"max_new_tokens": 40, "num_return_sequences": 2, "do_sample": True, "top_k": 50}

# Fonksiyonlar
def generate_standard():
    prompt = f"Problem: {problem}\nSolution:"
    inputs = tokenizer.encode(prompt, return_tensors="pt").to(device)
    outs = model.generate(inputs, **standard_kwargs)
    return [tokenizer.decode(o, skip_special_tokens=True).replace(prompt, "").strip() for o in outs]

def generate_cot():
    prompt = (
        f"Problem: {problem}\n"
        "Let's think step by step. Provide each reasoning step, then the final solution.\n"
        "Answer:"
    )
    inputs = tokenizer.encode(prompt, return_tensors="pt").to(device)
    outs = model.generate(inputs, **cot_kwargs)
    return [tokenizer.decode(o, skip_special_tokens=True).replace(prompt, "").strip() for o in outs]

def expand_thoughts(prefix):
    prompt = f"Problem: {problem}\nThoughts so far: {prefix}\nNext thought:"
    inputs = tokenizer.encode(prompt, return_tensors="pt").to(device)
    outs = model.generate(inputs, **tot_expand_kwargs)
    texts = [tokenizer.decode(o, skip_special_tokens=True) for o in outs]
    return [t.split("Next thought:")[-1].strip() for t in texts]

def generate_tot():
    paths = []
    first = expand_thoughts("")
    for t1 in first:
        second = expand_thoughts(t1)
        for t2 in second:
            paths.append([t1, t2])
    solutions = []
    for path in paths:
        prefix = " | ".join(path)
        prompt = f"Problem: {problem}\nThought path: {prefix}\nFinal solution:"
        inputs = tokenizer.encode(prompt, return_tensors="pt").to(device)
        # DÜZELTME: Buradaki hard-coded 'max_length' de 'max_new_tokens' olarak değiştirildi.
        out = model.generate(inputs, max_new_tokens=60, do_sample=True, top_k=50)
        sol = tokenizer.decode(out[0], skip_special_tokens=True).split("Final solution:")[-1].strip()
        solutions.append(sol)
    return solutions

# Üretim ve yazdırma
standard_sols = generate_standard()
cot_sols = generate_cot()
tot_sols = generate_tot()

def is_valid(sol):
    return ("6 stones" in sol) and ("3 stones" in sol)

print("=== Standart ===")
for i, s in enumerate(standard_sols,1):
    print(f"{i}. {s} -> Valid: {is_valid(s)}")

print("\n=== CoT ===")
for i, s in enumerate(cot_sols,1):
    print(f"{i}. {s} -> Valid: {is_valid(s)}")

print("\n=== Tree-of-Thoughts ===")
for i, s in enumerate(tot_sols,1):
    print(f"{i}. {s} -> Valid: {is_valid(s)}")

print("\n=== Summary ===")
print(f"Standard valid: {sum(is_valid(s) for s in standard_sols)}/{len(standard_sols)}")
print(f"CoT valid:      {sum(is_valid(s) for s in cot_sols)}/{len(cot_sols)}")
print(f"TOT valid:      {sum(is_valid(s) for s in tot_sols)}/{len(tot_sols)}")


=== Standart ===
1. There are 3 cups with 6 stones each. The 1st cup is 6 cups, moves 1 stone from there, then moves 3 stones to the next cup. Then, one stone from the 6th cup moves to the 8th cup. -> Valid: True
2. One cup starts with 8 stones. Each of the other cups starts with 3 stones each. Once you have 18 stones, your first move is to put 5 in the cup with 3 stones. The other two cups each start with 3 stones. The 3 -> Valid: False
3. One Cup is the tallest with 5 stones. Next are the two cups with the most stones: The first with 3 stone and is taller at 4 stones, where the second is shorter. The next 2 are the most granaries in their -> Valid: False
4. The green cup has 6 stones so it moves, so the two opposite cups have 6 and 3 stones each, which move. The blue cup has 3 stones per cup so it moves, so the two opposite cups have 3 and 5 stones each, which -> Valid: True

=== CoT ===
1. The first cup has 6 stones because 6 - 3 = 6. The second cup has 8 stones because 6 - 3 = 8. T

Bu çıktıda gördüğünüz gibi, üç yöntemle de (“Standart”, “Chain-of-Thought” ve “Tree-of-Thoughts”) üretilen dört’er çözümün tamamı bizim çok basit “6 stones” + “3 stones” kontrolümüzden geçemedi (0/4 geçerli). Bu birkaç şey söylüyor:

Modelin kapasitesi: GPT-2 küçük bir model; çok adımlı, sayısal mantık gerektiren bir problemi çözmekte yetersiz kalıyor. Dolayısıyla ne doğrudan sorduğunuzda, ne “adım adım düşün” talimatıyla, ne de dallanıp seçerek (TOT) düşündürdüğünüzde, hâlâ tutarlı bir “6 taşı bir kovaya, diğerlerinde 3’er taş bırak” çözümü üretemiyor.

Prompt mühendisliği sınırları:

Standart prompt: Model tamamen “yapay” veya alakasız metinler (futbol, su karıştırma reçetesi vs.) üretti.

CoT prompt: Model “adımları” betimlemeye çalışsa da, anlamsal olarak hâlâ problemin taş diziliminden kopuk, rastgele adımlar anlatıyor.

TOT prompt: Dallanmış düşünce adımlarını bile daha önce benzer metinlerden (liderlik, futbol, tarif vs.) türetip, sonunda yine alakasız bir “çözüm” sundu.

Simple validity check’in anlamı:
Bizim is_valid fonksiyonumuz “üretilen metinde mutlaka ‘6 stones’ AND ‘3 stones’ geçmeli” diye bakıyordu. Burada hiçbir çıkışta bu ifadeler birlikte görülmediği için — ki aslında birçoğu taş kavramından bile uzak metinler ürettiği için — tümü False döndü.

Nihai yorum:

Küçük modeller üzerinde CoT veya TOT teknikleri bile her zaman mucize yaratmıyor. Modelin kendi kapasitesi ve daha güçlü sayısal/işlemsel yetenekleri kısıtlıysa, prompt engineering tek başına yeterli değil.

Bu örnek, Tree-of-Thoughts’un gücünü göstermeye yönelik bir demo olsa da, altta yatan model yeterince güçlü değilse dallanıp değerlendirmenin de işe yaramadığını görüyoruz.

Gerçekten somut fayda görmek için, ya daha büyük bir model (örn. GPT-3.5/4 ya da FLAN-T5-XL) ya da sayısal yetenekleri daha kuvvetli bir LLM tercih etmek; ya da çözümü kod içinde açıkça sayısal işlemler yapan bir paketle (örneğin Python’da hesaplamayı biz yaptırıp sonucu modelle açıklatmak gibi) hibrit bir yaklaşım geliştirmek gerekebilir.

Özetle, “hiç düşünme”, “CoT” ve “TOT” karşılaştırması göstermek güzel bir fikir, ama başarıyı gerçekten görmek için arka plandaki modelin de yeterli kapasitede olması şart.